In [ ]:
!pip install -qq datasets transformers[sentencepiece]
!pip install rouge_score -qq

     |████████████████████████████████| 311 kB 5.0 MB/s 
     |████████████████████████████████| 3.4 MB 86.0 MB/s 
     |████████████████████████████████| 243 kB 90.5 MB/s 
     |████████████████████████████████| 1.1 MB 81.1 MB/s 
     |████████████████████████████████| 67 kB 6.1 MB/s 
     |████████████████████████████████| 133 kB 87.9 MB/s 
     |████████████████████████████████| 144 kB 87.9 MB/s 
     |████████████████████████████████| 271 kB 93.7 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 3.3 MB 75.3 MB/s 
     |████████████████████████████████| 596 kB 77.0 MB/s 
     |████████████████████████████████| 895 kB 73.5 MB/s 
     |████████████████████████████████| 1.2 MB 54.4 MB/s 


In [ ]:
import glob
import concurrent.futures
import pandas as pd
from transformers import pipeline
import numpy as np
from datasets import load_metric

##Load test dataset

In [ ]:
!wget 'https://github.com/ThanhChinhBK/vietnews/archive/master.zip'
!unzip 'master.zip'

In [ ]:
def listPaths(path):
  pathfiles = list()
  for pathfile in glob.glob(path):
    pathfiles.append(pathfile)
  return pathfiles

test_paths = listPaths('/content/vietnews-master/data/test_tokenized/*')

def read_content(pathfile):
  """
  Input: Path of txt file
  Output: A dictionary has keys 'original' and 'summary'
  """
  with open(pathfile) as f:
    rows  = f.readlines()
    original = ' '.join(''.join(rows[4:]).split('\n'))
    summary = ' '.join(rows[2].split('\n'))
          
  return {'file' : pathfile,
          'original': original, 
          'summary': summary}


def get_dataframe(pathfiles):
  with concurrent.futures.ProcessPoolExecutor() as executor:
    data = executor.map(read_content, pathfiles)
  
  # Make blank dataframe
  data_df = list()
  for d in data:
    data_df.append(d)
  data_df = pd.DataFrame(data_df)
  data_df.dropna(inplace = True)
  data_df = data_df.sample(frac=1).reset_index(drop=True)

  return data_df         

In [ ]:
test_df = get_dataframe(test_paths)

In [ ]:
test_df.head(3)

,file,original,summary
0,/content/vietnews-master/data/test_tokenized/0...,Vợ_chồng Thủ_tướng Singapore Lý_Hiển_Long và H...,"Hà_Tinh , phu_nhân Thủ_tướng Singapore Lý_Hiển..."
1,/content/vietnews-master/data/test_tokenized/0...,"Theo nguồn_tin riêng của VnExpress , cơ_quan a...",Tất_cả tuyến đường từ sân_bay Nội_Bài về trung...
2,/content/vietnews-master/data/test_tokenized/0...,Một vụ nổ đã làm rung_chuyển trung_tâm dẫn khí...,Một vụ nổ chưa rõ nguyên_do ở Thủ_đô nước Áo v...


In [ ]:
test_df['original'][0]

'Vợ_chồng Thủ_tướng Singapore Lý_Hiển_Long và Hà_Tinh ( phải ) . " Hầu_hết , nếu không muốn nói là ở tất_cả các quốc_gia khác , lãnh_đạo sẽ có nhiều đặc_quyền trong nhiệm_kỳ như quản_gia , thợ làm tóc , các chuyến bay miễn_phí trên các hãng hàng_không quốc_gia , thậm_chí có cả kỳ nghỉ dành cho gia_đình và nhiều lãnh_đạo Mỹ còn được hưởng các đặc_quyền sau khi đã kết_thúc nhiệm_kỳ " , bà Hà viết trên tài_khoản mạng xã_hội Facebook ngày 21/8 . Tuyên_bố của phu_nhân thủ_tướng Singapore được cho là nhắm đến bài viết trên trang blog Seedly , trong đó so_sánh lương của ông Lý với Tổng_thống Mỹ Donald_Trump , trưởng đặc_khu hành_chính Hong_Kong_Carrie_Lam và những lãnh_đạo khác . Bà Hà giải_thích rằng lãnh_đạo là những người " có phẩm_chất xuất_sắc " và không nên lợi_dụng điều đó mà " trả lương thấp cho họ hoặc yêu_cầu họ phải như các vị thánh " . Tháng 9 năm_ngoái , trang_web của chính_phủ Singapore công_khai mức lương của ông Lý là 2,2 triệu đô_la Singapore ( 1,6 triệu USD ) . Trang này tuy

## model

In [ ]:
rouge_score = load_metric("rouge")

Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

In [ ]:
model_name = "haotieu/vietnamese-summarization"

In [ ]:
from transformers import MBartForConditionalGeneration
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

def old_summarization_pipeline(df):
    input_ids = tokenizer.encode_plus(df['original'], return_tensors='pt', max_length=512, truncation=True, padding=True)['input_ids']
    summary_ids = model.generate(input_ids, max_length=128)
    summaries = [tokenizer.decode(s, skip_special_tokens=True) for s in summary_ids]
    return summaries

Downloading:   0%|          | 0.00/310 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/926 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57G [00:00<?, ?B/s]

In [ ]:
test_df = test_df.sample(1300)

In [ ]:
predictions = [old_summarization_pipeline(test_df.iloc[i])[0] for i in range(len(test_df))]

In [ ]:
references = list(test_df['summary'])

In [ ]:
rouge_score.compute(predictions=predictions,references=references)

{'rouge1': AggregateScore(low=Score(precision=0.6421099015228792, recall=0.5827788473933142, fmeasure=0.5994822492420822), mid=Score(precision=0.6488294631326927, recall=0.5903851036033321, fmeasure=0.6046048126363871), high=Score(precision=0.6551427612497637, recall=0.5975568788618779, fmeasure=0.6099988147969468)),
 'rouge2': AggregateScore(low=Score(precision=0.30937329532484364, recall=0.28219930379342123, fmeasure=0.28901008979453263), mid=Score(precision=0.3184830324441066, recall=0.2906253483421962, fmeasure=0.2972045723062174), high=Score(precision=0.32788381465864164, recall=0.2995529341979984, fmeasure=0.3059658609364611)),
 'rougeL': AggregateScore(low=Score(precision=0.4218948602089806, recall=0.3832056068336277, fmeasure=0.39319662200988503), mid=Score(precision=0.4293836223217873, recall=0.3907846108396068, fmeasure=0.4002650527478388), high=Score(precision=0.4366476841915338, recall=0.39836214576725526, fmeasure=0.406886113772301)),
 'rougeLsum': AggregateScore(low=Score

In [ ]:
rouge_score.compute(predictions=predictions,references=references)['rouge1'].mid.fmeasure

0.6046048126363871

In [ ]:
rouge_score.compute(predictions=predictions,references=references)['rouge2'].mid.fmeasure

0.2972045723062174

In [ ]:
rouge_score.compute(predictions=predictions,references=references)['rougeL'].mid.fmeasure

0.4002650527478388

## result dataframe

In [ ]:
result_df = pd.DataFrame({'summary':references, 'predictions':predictions})

In [ ]:
print(result_df.iloc[0]['summary'])

Chuỗi cửa_hàng tiện_lợi FamilyMart ngày 5/8 xin_lỗi sau khi video nhiều con chuột chạy trên kệ hàng ở Shibuya , Tokyo xuất_hiện trên mạng xã_hội . 


In [ ]:
print(result_df.iloc[0]['predictions'])

Chuỗi siêu_thị FamilyMart ở Tokyo phải đóng_cửa sau khi video ghi lại cảnh chuột bò trên nóc tủ_lạnh trong một cửa_hàng.


In [ ]:
result_df.to_csv("resulr_test_summarization")